<a href="https://colab.research.google.com/github/nachodelis/deepLearning/blob/master/Generaci%C3%B3n_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generación de textos de terror

##Introducción

En este trabajo se ha utilizado un enfoque basado en redes neuronales recurrentes para generar texto a partir de un conjunto de datos de texto existente. El objetivo de este modelo es generar texto que sea coherente y tenga sentido, similar al texto que se encuentra en el conjunto de datos de entrenamiento.

##Código

Primero importamos las librerías

In [1]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

La primera parte del código implica cargar los datos en lotes (o "batches") de 10 filas y luego crear un vocabulario de palabras únicas y asignar un índice único a cada palabra en el conjunto de datos. Se utiliza la función StringLookup de la librería preprocessing de TensorFlow para codificar las palabras del conjunto de datos.

A continuación, se convierten todas las letras a minúsculas y se eliminan los signos de puntuación. Después, se reemplaza cada palabra en el conjunto de datos con su índice correspondiente utilizando el codificador de palabras.

Luego se ajusta la longitud de cada secuencia de palabras a la longitud máxima, llenando con ceros al final de la secuencia.

Se definen los parámetros de la red neuronal recurrente, como la dimensión del embedding y las unidades RNN, y se define la arquitectura de la red neuronal recurrente en sí. El modelo se compila y se entrena en el conjunto de datos cargado previamente.

El modelo predecirá la siguiente palabra en una oración en función de las palabras anteriores.

In [ ]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

# cargamos los datos en batches de 10 filas
chunksize = 10
for chunk in pd.read_csv('terror.csv', delimiter=None, error_bad_lines=False, nrows=200, chunksize=chunksize):
    
    # crea un vocabulario de palabras únicas y asigna un índice único a cada palabra
    vocab_size = 10000
    word_counts = pd.Series(' '.join(chunk['body']).split()).value_counts()
    vocab = list(word_counts.iloc[:vocab_size - 1].index)

    # crea una capa de codificación de palabras usando preprocessing.StringLookup
    encoder = preprocessing.StringLookup(vocabulary=vocab, mask_token=None)

    # convierte todas las letras a minúsculas y elimina signos de puntuación
    chunk['body'] = chunk['body'].apply(lambda x: x.lower().translate(str.maketrans('', '', string.punctuation)))

    # reemplaza cada palabra en el conjunto de datos con su índice correspondiente usando el codificador de palabras
    chunk['body'] = chunk['body'].apply(lambda x: np.array(encoder(x.split())))

    # ajustar la longitud de cada secuencia a la longitud máxima llenando con ceros al final de la secuencia
    max_length = max(chunk['body'].apply(len))
    chunk['body'] = chunk['body'].apply(lambda x: np.pad(x, (0, max_length - len(x)), 'constant'))

    # define los parámetros de la red neuronal recurrente
    embedding_dim = 256
    rnn_units = 100

    # define la arquitectura de la red neuronal recurrente
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=embedding_dim),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(len(encoder.get_vocabulary())+1)
])

    # compila el modelo
    model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer='adam')

    # entrena el modelo
    batch_size = 5
    sequence_length = max_length
    input_data = np.array(chunk['body'].tolist())[:, :-1]  # Todas las palabras menos la última
    output_data = np.array(chunk['body'].tolist())[:, 1:]  # La siguiente palabra para cada palabra de entrada
    history = model.fit(input_data, output_data, batch_size=batch_size, epochs=50)

<ipython-input-2-106e71fb8d45>:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  for chunk in pd.read_csv('terror.csv', delimiter=None, error_bad_lines=False, nrows=200, chunksize=chunksize):


Epoch 1/50
2/2 [==============================] - 28s 12s/step - loss: 8.7100
Epoch 2/50
2/2 [==============================] - 23s 11s/step - loss: 8.6803
Epoch 3/50
2/2 [==============================] - 24s 12s/step - loss: 8.6262
Epoch 4/50
2/2 [==============================] - 24s 12s/step - loss: 8.4962
Epoch 5/50
2/2 [==============================] - 24s 12s/step - loss: 8.2536
Epoch 6/50
2/2 [==============================] - 24s 12s/step - loss: 7.9549
Epoch 7/50
2/2 [==============================] - 24s 12s/step - loss: 7.6472
Epoch 8/50
2/2 [==============================] - 25s 13s/step - loss: 7.3545
Epoch 9/50
2/2 [==============================] - 23s 12s/step - loss: 7.0414
Epoch 10/50
2/2 [==============================] - 24s 12s/step - loss: 6.7173
Epoch 11/50
2/2 [==============================] - 24s 12s/step - loss: 6.3730
Epoch 12/50
2/2 [==============================] - 24s 12s/step - loss: 6.0508
Epoch 13/50
2/2 [==============================] - 24s 12s/st

La segunda parte del código utiliza el modelo de red neuronal recurrente entrenado previamente para generar texto a partir de una secuencia de palabras iniciales. En este caso, se especifica una lista de tres palabras de inicio, y se establece la variable num_words en 50 para generar un total de 50 palabras adicionales.

Primero, las palabras de inicio se convierten en una secuencia de índices utilizando el codificador de palabras creado previamente. Esto se hace para que el modelo pueda procesar las palabras como entrada y generar nuevas palabras como salida.

Luego, se utiliza un bucle for para generar num_words palabras adicionales. En cada iteración del bucle, se predice la palabra siguiente dada la secuencia actual de palabras utilizando el modelo de red neuronal recurrente previamente entrenado. Esto se hace utilizando el método predict del modelo para obtener las predicciones de salida.

La predicción se convierte en un índice de palabra utilizando el método tf.random.categorical, que selecciona un índice al azar de las distribuciones de probabilidad de salida. La palabra predicha se agrega a la secuencia de entrada actual, y el proceso se repite para generar la siguiente palabra.

Finalmente, la secuencia de índices de palabras generada se convierte de nuevo en palabras utilizando el codificador de palabras invertido, y se muestra el resultado utilizando el método join de Python para unir las palabras en una cadena de texto.

In [ ]:
# Generar texto de tres palabras de inicio
start_words = ['this', 'job', 'is']
num_words = 50

# Convierte las palabras de inicio en una secuencia de índices usando el codificador de palabras
start_sequence = np.array(encoder(start_words)).reshape(1, -1)

# Genera num_words palabras adicionales usando el modelo
for i in range(num_words):
    # Predice la palabra siguiente dada la secuencia actual de palabras
    predicted_logits = model.predict(start_sequence)[:, -1, :]
    predicted_id = tf.random.categorical(predicted_logits, num_samples=1)
    # Añade la palabra predicha a la secuencia de entrada
    start_sequence = np.concatenate([start_sequence, predicted_id], axis=-1)

# Convierte la secuencia de índices en palabras y muestra el resultado
predicted_words = tf.keras.layers.experimental.preprocessing.StringLookup(
                    vocabulary=encoder.get_vocabulary(), invert=True)(start_sequence)[0].numpy()
predicted_words = predicted_words.reshape((-1,))

predicted_words = [word.decode('utf-8') for word in predicted_words]
print(' '.join(predicted_words))


##Conclusión


En conclusión, el código muestra cómo entrenar un modelo de lenguaje utilizando redes neuronales recurrentes para generar texto a partir de un corpus de datos. Utilizando el conjunto de datos de terror como ejemplo, se utiliza el preprocesamiento de datos, la codificación de palabras y la definición de la arquitectura de la red neuronal para entrenar el modelo. El modelo se usa para generar texto a partir de una secuencia de palabras de inicio dadas y se muestra cómo se puede convertir la secuencia de índices generada en palabras para mostrar el resultado final.